# Projeto: Análise de Sentimentos em Discursos Políticos

<font color='black'>**Parte 1: Coleta de dados**</font><br>
<font color='gray'>Parte 2: Limpeza de dados</font><br>
<font color='gray'>Parte 3: Modelagem e Análise de Sentimentos</font>

![Alt text](./images/web-scraping-1.svg)

Esta é a primeira parte deste projeto, no qual vamos obter os discursos políticos do presidente atual do Brasil, diretamente de suas fontes oficiais, através de um `Web Scraping`, no site público dos [discursos](https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos) do governo.

Nosso objetivo é coletar as transcrições de seus discursos oficiais em 2019 e 2020.

Vamos coletar os dados!

## Importando as bibliotecas

Iniciaremos importando as bibliotecas que serão utilizadas. As principais delas são:
* **BeautifulSoup:** é uma biblioteca Python de extração de dados de arquivos HTML e XML. Ele cria uma árvore de análise para páginas analisadas que podem ser usadas para extrair dados de HTML, o que é útil para web scraping.
* **Selenium:** usado para testar aplicações web pelo browser de forma automatizada.

In [1]:
# Manipulação de dados
import os
import re
import sys
import time
import string
import pickle
import requests
import datetime
import numpy as np
import pandas as pd

# Web Scraping
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Definimos o parâmetro abaixo para evitar o erro: RecursionError: maximum recursion depth exceeded
# Ao gravar os resultados em disco
sys.setrecursionlimit(50000)

Vamos definir o **chromedriver** para abrir o navegador e executar as instruções automaticamente. O chromedriver deve ser da mesma versão do navegador google chrome instalado na máquina e ser colocado na mesma pasta do projeto.

In [2]:
# utilizar chromedriver
chromedriver = "./chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

Então vamos começar criando uma lista vazia para armazenar as urls para coleta. Neste caso, já analisado previamente, as páginas seguem um padrão com o número no final, múltiplos de 30, portanto, podemos criar um loop para inserirmos o número de cada página. Lembrando que nem todas as páginas são assim e dependendo da página deverá analisar a melhor forma de definir.

In [3]:
# cria um lista vazia para receber as urls
lista_urls = []

# loop para criar a lista
for i in range(0, 271, 30):
    full_url = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?' + 'b_start:int=' + str(i)
    lista_urls.append(full_url)

Vamos dar uma olhada na lista, no nosso caso são 10 urls.

In [4]:
# visualizando as listas
lista_urls

['https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=0',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=30',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=60',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=90',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=120',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=150',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=180',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=210',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=240',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=270']

Agora vamos definir um contador, este contador é o número de páginas que queremos coletar, a princípio podemos coletar um pouco e aumentarmos se desejamos. A quantidade de páginas para coleta está diretamente ligada ao tempo da coleta, então vamos iniciar com 5 páginas.

In [5]:
# criando o contador
contador = 5

As URLs definidas anteriormente são os locais onde contém os discursos, cada URL contém 30 links, correspondentes à 30 discursos, que serão as próximas coletas.

Criaremos agora uma função para começar as coletas.<br>
Observação: quando vamos coletar várias páginas, não devemos deixar o loop rodando direto, ou seja, deve-se ter uma pausa antes da página ir para a próxima, essa boa prática deve ser utilizada, pois se não o servidor da página pode sobrecarregar e seremos derrubados. Na função abaixo colocamos um tempo de 5s `time.sleep(5)`.

In [6]:
# criando a função para scraping
def scraping_urls(urls):
    
    # definindo o driver
    driver = webdriver.Chrome("./chromedriver")
    
    # lista para receber o resultado
    soup_list = []
    
    # contador para o loop
    count = 0
    
    # iniciando a contagem do tempo
    start_time = time.time()
    
    # loop pelas urls
    for i in urls:
        if count < contador:
            driver.get(i)
            driver.refresh()
            time.sleep(5)
            soup_list.append(BeautifulSoup(driver.page_source, 'html.parser'))
        count += 1  
    driver.close()
    
    # tempo decorrido do loop
    current_time = time.time()
    elapsed_time = current_time - start_time
    print("Tempo total da coleta: " + str(int(elapsed_time))  + " segundos.")
    
    # retornando o resultado da função
    return soup_list

Com a função criada, então vamos executá-la e em seguida visualizar o resultado da coleta.

In [7]:
# executando a função
soups = scraping_urls(lista_urls)

Tempo total da coleta: 51 segundos.


In [1]:
# visualizando
# soups

O resultado é o código fonte, html da página, agora utilizaremos os métodos da biblioteca `BeautifulSoup` e eventualmente podemos utilizar também as expressões regulares `regex`.

Mas antes vamos dar uma olhada, à quantas páginas correspondem esses códigos com a função `len()`.

In [9]:
len(soups)

5

Agora criaremos a função que extrairá as URLs de cada discurso e cada URL será armazenada em uma lista.

Mas como vamos descobrir em qual parte do código HTML constam essas URLs?<br>
Deve-se procurar pelas *tags* e *classes* que formam a página, uma forma mais fácil, pelo google chrome clicando com o botão direito em algum lugar da página, clique na opção *inspecionar*,  ai abrirá a janela de desenvolvedor, com isso poderá ir com a seta em cima do link na página e será direcionada para esta parte do código, então você insere nos métodos da biblioteca *BeautifulSoup*. Vá aos poucos e assim vai aprendendo em que momentos colocará *tag* e *classes* para extrair o resultado.

In [10]:
# criando a função
def extrai_links(soup_object):
    
    # lista para receber o resultado
    links_list = []
    
    # loop pela código da pagina
    for s in soup_object:
        links = s.find_all('a', {'class':'summary url'})
        for i in links:
            full_link = i['href']
            links_list.append(full_link)
            
    # retorna o resultado
    return links_list

Vamos excutar a função e dar uma olhada no resultado, do que sobrou do código.

In [11]:
link_list = extrai_links(soups)
link_list[0:4]

['https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-no-lancamento-dos-programas-codex-e-super-br-e-8o-revogacao-palacio-do-planalto',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-na-cupula-do-g20-videoconferencia-palacio-do-planalto-1',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-na-cupula-do-g20-videoconferencia-palacio-do-planalto',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-entrega-de-titulos-de-propriedade-rural-em-goias-flores-de-goias']

Era isso mesmo que queríamos! vamos dar uma olhada em quantos links, no nosso caso, são 5 páginas de coletas com 30 discursos, dando um total de 150 links de discursos.

In [12]:
len(link_list)

150

Vamos salvar esses links, caso precisarmos mais tarde, não será necessário rodar novamente, vamos utilizar a extensão `pickle`.

In [13]:
# salvando em disco
with open('dados/db_links.pickle', 'wb') as to_write:
    pickle.dump(link_list, to_write)

Para finalizarmos as coletas, nos falta a parte mais importante que são os discursos, também criaremos uma função para isso.

In [14]:
# criando a função que extrai o discurso político do código fonte
def extrai_discurso(urls):
    
    # instancia o driver do chromedriver
    driver = webdriver.Chrome(chromedriver)
    
    # lista vazia para receber o resultado
    doc_source = []
    
    # inicia o tempo para o loop
    start_time = time.time()
    
    # loop para coleta dos dados
    for i in urls:
        driver.get(i)
        time.sleep(5)
        doc_source.append(BeautifulSoup(driver.page_source, 'html.parser'))
    driver.close()

    # tempo decorrido do loop
    current_time = time.time()
    elapsed_time = current_time - start_time
    print("Tempo total da coleta dos discursos: " + str(int(elapsed_time))  + " segundos.")
    
    # retorna o resultado da função
    return doc_source

Executando a função!

In [15]:
discursos = extrai_discurso(link_list)

Tempo total da coleta dos discursos: 1049 segundos.


Por fim vamos salvar o código fonte HTML da forma que coletamos e a extração dos discursos e limpeza ficarão para o próximo *notebook*.

In [16]:
# salvando em disco
with open('dados/db_discursos.pickle', 'wb') as to_write:
    pickle.dump(discursos, to_write)